<a href="https://colab.research.google.com/github/Asadityasingh/DLP/blob/main/Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
from datasets import load_dataset
ds = load_dataset('bookcorpus',split='all')
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

bookcorpus.py:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

The repository for bookcorpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bookcorpus.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 74004228
})

In [25]:
from tokenizers.normalizers import Lowercase
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.trainers import BpeTrainer

In [26]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Set normalizers and pre-tokenizers
tokenizer.normalizer = Lowercase()
tokenizer.pre_tokenizer = Whitespace()


In [30]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=32000,special_tokens=["[PAD]","[UNK]"],continuing_subword_prefix='##')

In [31]:
def get_examples(batch_size=1000):
  for i in range(0, len(ds), batch_size):
    yield ds[i: i+ batch_size]['text']

In [32]:
tokenizer.train_from_iterator(get_examples(batch_size=10000),trainer=trainer,length=len(ds))

In [33]:
sample = ds[0]['text']
print(f'sample: {sample}')
encoding = tokenizer.encode(sample)
encoding

sample: usually , he would be tearing around the living room , playing with his toys .


Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [34]:
token_ids = encoding.ids
tokens = encoding.tokens
type_ids = encoding.type_ids
attention_mask = encoding.attention_mask

In [35]:
from tokenizers.tools import EncodingVisualizer
visualizer = EncodingVisualizer(tokenizer=tokenizer)
visualizer(text=sample)

In [36]:
import pandas as pd
out_dict = {'tokens': tokens,'ids':token_ids,'type_ids': type_ids,'attention_mask': attention_mask}
df = pd.DataFrame.from_dict(out_dict)
df

,tokens,ids,type_ids,attention_mask
0,usually,2462,0,1
1,",",19,0,1
2,he,149,0,1
3,would,277,0,1
4,be,162,0,1
5,tearing,6456,0,1
6,around,422,0,1
7,the,131,0,1
8,living,1559,0,1
9,room,536,0,1


Batch Encoding

In [38]:
sample = ds[0:4]['text']
sample

['usually , he would be tearing around the living room , playing with his toys .',
 'but just one look at a minion sent him practically catatonic .',
 "that had been megan 's plan when she got him dressed earlier .",
 "he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older ."]

In [39]:
batch_encoding = tokenizer.encode_batch(sample)
print(batch_encoding)

[Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=42, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]


In [40]:
tokenizer.enable_padding(direction = 'right',
                         pad_id = 0,
                         pad_type_id = 0,
                         pad_token = '[PAD]',
                         length = None,
                         pad_to_multiple_of=None
                         )
tokenizer.enable_truncation(max_length=512)

In [42]:
batch_encoding = tokenizer.encode_batch(sample)
print(batch_encoding)

[Encoding(num_tokens=42, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=42, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=42, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=42, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]


In [45]:
 text = "All this is so simple to do in HF :) :> ."
 tokenizer.encode(text).tokens

['all',
 'this',
 'is',
 'so',
 'simple',
 'to',
 'do',
 'in',
 'h',
 '##f',
 ':',
 '[UNK]',
 ':',
 '[UNK]',
 '.']

In [47]:
text

'All this is so simple to do in HF :) :> .'

In [48]:
#Save Tokenizer
tokenizer.save('hopper.json')

In [49]:

import json
with open('hopper.json','r') as file:
  json_data = json.load(file)
json_data

{'version': '1.0',
 'truncation': {'direction': 'Right',
  'max_length': 512,
  'strategy': 'LongestFirst',
  'stride': 0},
 'padding': {'strategy': 'BatchLongest',
  'direction': 'Right',
  'pad_to_multiple_of': None,
  'pad_id': 0,
  'pad_type_id': 0,
  'pad_token': '[PAD]'},
 'added_tokens': [{'id': 0,
   'content': '[PAD]',
   'single_word': False,
   'lstrip': False,
   'rstrip': False,
   'normalized': False,
   'special': True},
  {'id': 1,
   'content': '[UNK]',
   'single_word': False,
   'lstrip': False,
   'rstrip': False,
   'normalized': False,
   'special': True}],
 'normalizer': {'type': 'Lowercase'},
 'pre_tokenizer': {'type': 'Whitespace'},
 'post_processor': None,
 'decoder': None,
 'model': {'type': 'BPE',
  'dropout': None,
  'unk_token': '[UNK]',
  'continuing_subword_prefix': '##',
  'end_of_word_suffix': None,
  'fuse_unk': False,
  'byte_fallback': False,
  'ignore_merges': False,
  'vocab': {'[PAD]': 0,
   '[UNK]': 1,
   '\x13': 2,
   '\x14': 3,
   '\x18': 4,
 

In [50]:
# Load tokenizer
trained_tokenizer = Tokenizer(BPE())
trained_tokenizer = trained_tokenizer.from_file('hopper.json')

In [51]:
tokens = trained_tokenizer.encode(text).tokens
print(tokens)

['all', 'this', 'is', 'so', 'simple', 'to', 'do', 'in', 'h', '##f', ':', '[UNK]', ':', '[UNK]', '.']


BERT Tokenizer

In [52]:
bert_tokenizer = Tokenizer(BPE(unk_token='[UNK]'))
bert_tokenizer.normalizer = Lowercase()
bert_tokenizer.pre_tokenizer = Whitespace()
bart_trainer = BpeTrainer(vocab_size = 32000,
                          special_tokens=["[PAD]","[CLS]","[SEP]","[MASK]"],
                          continuing_subword_prefix='##')

In [53]:
from tokenizers.processors import TemplateProcessing

In [55]:
bert_tokenizer.post_processor = TemplateProcessing(single="[CLS] $0 [SEP]",
                                                   pair = "[CLS] $A [SEP] $B:1",
                                                   special_tokens=[("[CLS]",2),("[SEP]",3)],)

In [56]:
bert_tokenizer.post_processor

TemplateProcessing(single=[SpecialToken(id="[CLS]", type_id=0), Sequence(id=A, type_id=0), SpecialToken(id="[SEP]", type_id=0)], pair=[SpecialToken(id="[CLS]", type_id=0), Sequence(id=A, type_id=0), SpecialToken(id="[SEP]", type_id=0), Sequence(id=B, type_id=1)], special_tokens={"[CLS]":SpecialToken(id="[CLS]", ids=[2], tokens=["[CLS]"]), "[SEP]":SpecialToken(id="[SEP]", ids=[3], tokens=["[SEP]"])})

In [57]:
bert_tokenizer.train_from_iterator(get_examples(batch_size=10000),trainer=bart_trainer,length=len(ds))

In [62]:
from pprint import pprint

In [66]:
text = "All these are so simple to do in HF, Let's do more"
encoded = bert_tokenizer.encode(text)
tokens = encoded.tokens
ids = encoded.ids
out_dict = {'tokens': tokens, 'ids':ids}
pprint(out_dict,depth=2,compact=True)


{'ids': [2, 269, 955, 335, 230, 2533, 140, 205, 156, 55, 103, 21, 461, 16, 66,
         205, 386, 3],
 'tokens': ['[CLS]', 'all', 'these', 'are', 'so', 'simple', 'to', 'do', 'in',
            'h', '##f', ',', 'let', "'", 's', 'do', 'more', '[SEP]']}


In [69]:
text = "All these are so simple to do in HF, Let's do more"
pair = "we have a long way to go!"
encoded = bert_tokenizer.encode(text,pair)
tokens = encoded.tokens
ids = encoded.ids
out_dict = {'tokens': tokens,'ids':ids}
pprint(out_dict,depth=2,compact=True)

{'ids': [2, 269, 955, 335, 230, 2533, 140, 205, 156, 55, 103, 21, 461, 16, 66,
         205, 386, 3, 213, 249, 48, 489, 414, 140, 259, 11],
 'tokens': ['[CLS]', 'all', 'these', 'are', 'so', 'simple', 'to', 'do', 'in',
            'h', '##f', ',', 'let', "'", 's', 'do', 'more', '[SEP]', 'we',
            'have', 'a', 'long', 'way', 'to', 'go', '!']}


In [70]:
plain_tokens = bert_tokenizer.decode(ids)
print(plain_tokens)

all these are so simple to do in h ##f , let ' s do more we have a long way to go !


In [73]:
from tokenizers.decoders import WordPiece
bert_tokenizer.decoder = WordPiece(prefix='##')
plain_tokens = bert_tokenizer.decode(ids)
print(plain_tokens)

all these are so simple to do in hf, let ' s do more we have a long way to go!


PreTrainedTokenizer

In [74]:
from transformers import PreTrainedTokenizerFast

In [76]:
pt_tokenizer = PreTrainedTokenizerFast(tokenizer_file='hopper.json',
                                       unk_token='[UNK]',
                                       pad_token='[PAD]',
                                       model_input_names=["input_ids","token_type_ids","attention_mask"])

In [77]:
model_input = pt_tokenizer(text)
pprint(model_input,compact=True)

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [267, 953, 333, 228, 2531, 138, 203, 154, 53, 83, 19, 459, 14, 64,
               203, 384],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [78]:
model_input = pt_tokenizer(text,text_pair=pair)
pprint(model_input,compact=True)

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1],
 'input_ids': [267, 953, 333, 228, 2531, 138, 203, 154, 53, 83, 19, 459, 14, 64,
               203, 384, 211, 247, 46, 487, 412, 138, 257, 9],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
                    1, 1, 1, 1]}


In [80]:
batch_text = ['i like the book the Psychology of Money','I enjoyed watching the Transformers movie','oh! thanks for this']
model_input = pt_tokenizer(batch_text)
pprint(model_input,compact=True)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1]],
 'input_ids': [[54, 281, 131, 1701, 131, 19478, 153, 1564],
               [54, 4096, 1443, 131, 7744, 307, 3760],
               [772, 9, 1767, 200, 254]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]}


In [82]:
batch_text = ['i like the book the Psychology of Money','I enjoyed watching the Transformers movie','oh! thanks for this']
model_input = pt_tokenizer(batch_text,padding=True)
pprint(model_input,compact=True)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0],
                    [1, 1, 1, 1, 1, 0, 0, 0]],
 'input_ids': [[54, 281, 131, 1701, 131, 19478, 153, 1564],
               [54, 4096, 1443, 131, 7744, 307, 3760, 0],
               [772, 9, 1767, 200, 254, 0, 0, 0]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0]]}
